In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Loss functions and gradients
class Loss:
    @staticmethod
    def cross_entropy(y_true, y_pred):
        # Clip values to avoid log(0) issues
        epsilon = 1e-12
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
        loss = -np.sum(y_true * np.log(y_pred))
        return loss

    @staticmethod
    def cross_entropy_gradient(y_true, y_pred):
        return y_pred - y_true

# Activation functions and derivatives
class Activation:
    @staticmethod
    def relu(z):
        return np.maximum(0, z)

    @staticmethod
    def relu_derivative(z):
        return np.where(z > 0, 1, 0)

    @staticmethod
    def softmax(z):
        # Subtract max for numerical stability
        z -= np.max(z, axis=1, keepdims=True)
        exp_vals = np.exp(z)
        return exp_vals / np.sum(exp_vals, axis=1, keepdims=True)

    # For softmax used with cross-entropy, the derivative is taken care of in the loss gradient

# Fully connected layer definition
class Layer:
    def __init__(self, num_in, num_out, activation, activation_deriv, learning_rate=0.01):
        rng = np.random.default_rng(42)
        # He initialization for layers using ReLU
        self.weights = rng.standard_normal((num_in, num_out)) * np.sqrt(2 / num_in)
        self.biases = np.zeros((1, num_out))
        self.activation = activation
        self.activation_deriv = activation_deriv
        self.learning_rate = learning_rate
        self.inputs = None
        self.z = None

    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        return self.activation(self.z)

    def backward(self, dl_dy):
        # For layers with ReLU or similar activations
        if self.activation_deriv is not None:
            dl_dz = dl_dy * self.activation_deriv(self.z)
        else:
            dl_dz = dl_dy
        dl_dw = self.inputs.T @ dl_dz
        dl_db = np.sum(dl_dz, axis=0, keepdims=True)
        dl_dx = np.dot(dl_dz, self.weights.T)

        batch_size = self.inputs.shape[0]
        self.weights -= self.learning_rate * (dl_dw / batch_size)
        self.biases  -= self.learning_rate * (dl_db / batch_size)
        return dl_dx


In [ ]:
# Standard one-hot encoding function
def one_hot_encode(num_classes, y):
    return np.eye(num_classes)[y]

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,0,35.0,0,0,8.0500,S


In [ ]:
df = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

In [ ]:
df["Sex"] = df["Sex"].map({"male" : 0, "female": 1})

In [ ]:
df.dropna(axis=0, how="any", inplace=True)

In [ ]:
df["Embarked"] = df["Embarked"].map({"S": 0, "C":1 , "Q": 2})

In [ ]:
X = df.drop(columns=["Survived"]).values
y = df["Survived"].values

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [ ]:
y_train_encoded = one_hot_encode(2, y_train)
y_test_encoded = one_hot_encode(2, y_test)
# Define the model with two output neurons
model = [
    Layer(X_train.shape[1], 8, Activation.relu, Activation.relu_derivative),
    Layer(8, 8, Activation.relu, Activation.relu_derivative),
    Layer(8, 2, lambda x: x, lambda x: np.ones_like(x))
]

# Training loop
EPOCHS = 1000
BATCH_SIZE = 16

for epoch in range(EPOCHS):
    epoch_loss = 0
    num_batches = 0
    for batch_idx in range(0, len(X_train), BATCH_SIZE):
        X_batch = X_train[batch_idx:batch_idx+BATCH_SIZE]
        y_batch = y_train_encoded[batch_idx:batch_idx+BATCH_SIZE]

        # Forward propagation
        output = X_batch
        for layer in model:
            output = layer.forward(output)

        # Compute loss and its derivative
        loss = Loss.cross_entropy(y_batch, output)
        loss_der = Loss.cross_entropy_gradient(y_batch, output)

        # Backward propagation
        for layer in reversed(model):
            loss_der = layer.backward(loss_der)

        epoch_loss += np.mean(loss)
        num_batches += 1

    print(f"Epoch {epoch+1} loss: {epoch_loss / num_batches}")


Epoch 1 loss: 142.31318817227637
Epoch 2 loss: 34.65362090983652
Epoch 3 loss: 19.88474286274528
Epoch 4 loss: 17.034544603410712
Epoch 5 loss: 14.950383810154314
Epoch 6 loss: 14.47086487780345
Epoch 7 loss: 14.038791783307309
Epoch 8 loss: 13.687237349776337
Epoch 9 loss: 12.821593013014253
Epoch 10 loss: 12.580629388904669
Epoch 11 loss: 12.969425980924882
Epoch 12 loss: 12.752782260427182
Epoch 13 loss: 13.206617041126552
Epoch 14 loss: 13.010992410168951
Epoch 15 loss: 12.864835042782529
Epoch 16 loss: 13.306642254510688
Epoch 17 loss: 13.15893282784201
Epoch 18 loss: 13.029133142733013
Epoch 19 loss: 12.911564990452549
Epoch 20 loss: 12.800149409376107
Epoch 21 loss: 12.694857196106916
Epoch 22 loss: 12.606347225000807
Epoch 23 loss: 12.517940424626028
Epoch 24 loss: 12.435616066197994
Epoch 25 loss: 12.363134277793622
Epoch 26 loss: 12.30199718190059
Epoch 27 loss: 11.00327716440711
Epoch 28 loss: 10.900523336119669
Epoch 29 loss: 10.8250315398416
Epoch 30 loss: 10.7639020498709

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
def test_model(model, X_test, y_test_encoded, y_test):
    y_true = []
    y_pred = []

    for i in range(X_test.shape[0]):
        # Forward pass
        output = X_test[i]
        for layer in model:
            output = layer.forward(output)

        # Get predicted class (index of max probability)
        predicted_class = np.argmax(output)
        true_class = np.argmax(y_test_encoded[i])

        # Store values for metrics calculation
        y_true.append(true_class)
        y_pred.append(predicted_class)

    # Calculate accuracy
    accuracy = (np.array(y_true) == np.array(y_pred)).mean() * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

    # Calculate precision and recall for each class
    num_classes = y_test_encoded.shape[1]
    for cls in range(num_classes):
        precision = precision_score(y_true, y_pred, labels=[cls], average="macro", zero_division=0)
        recall = recall_score(y_true, y_pred, labels=[cls], average="macro", zero_division=0)
        print(f"Class {cls}: Precision = {precision:.2f}, Recall = {recall:.2f}")


In [ ]:
test_model(model, X_test, y_test_encoded, y_test)

Test Accuracy: 79.02%
Class 0: Precision = 0.78, Recall = 0.86
Class 1: Precision = 0.80, Recall = 0.70
